In [8]:
# import the necessary packages
import tensorflow
import keras
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import tensorflow as tf
import imutils
import time
import cv2
from os.path import dirname, join
import os
from keras.backend import expand_dims, repeat_elements
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import cv2
import time
output_notebook()


# load our serialized face detector model from disk
print("[INFO] loading face detector model...")

prototxtPath = "face_detector/deploy.prototxt"
weightsPath = "face_detector/res10_300x300_ssd_iter_140000.caffemodel"

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model("Models/3frame150CNNRNN.h5")

out = cv2.VideoWriter('CNNRNNmodeloutput1.mp4', -1, 10.0, (640,480))
print("done")

Loading BokehJS ...

[INFO] loading face detector model...
[INFO] loading face mask detector model...
done


In [9]:
frame_array = []

def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))
    
    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    faces = []
    locs = []
    preds = []
    
    # loop over the detections
    for i in range(0, detections.shape[2]):
        if detections[0, 0, i, 2] > 0.4:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            face = frame[startY:endY, startX:endX]
            #cv2.imshow("face",face)
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (150, 150))
            face = img_to_array(face)
            
            frameslist=[]
            count=0
            while count<3:
                frameslist.append(face)
                count=count+1
            faces1=np.array(frameslist)
            
            faces.append(faces1)
            locs.append((startX, startY, endX, endY))
        
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        #print(faces.shape)
        preds = maskNet.predict(faces)
        #print(preds)
        
    # return a 2-tuple of face locations and their corresponding
    return (locs, preds)



#maskNet = tf.keras.models.load_model((".h5"),custom_objects={'KerasLayer':hub.KerasLayer})
# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(0)
time.sleep(1)
ret, frame = cap.read()
frame = cv2.resize(frame, (640, 480))
frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
frame=cv2.flip(frame, 0) # because Bokeh flips vertically
width=640
height=480
p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
show(p, notebook_handle=True)
# loop over the frames from the video stream
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))
    
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        (x, y, w, h) = box
        (mask, withoutMask) = pred
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (w, h), color, 2)
    
    out.write(frame)
    frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    frame=cv2.flip(frame, 0)
    myImage.data_source.data['image']=[frame]
    push_notebook()
    #cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        out.release()
        break


# do a bit of cleanup
cap.release()
cv2.destroyAllWindows()


[INFO] starting video stream...


KeyboardInterrupt: 